In [ ]:
pip install ultralytics opencv-python numpy torch

In [ ]:
import os
os.chdir('/content')

current_directory = os.getcwd()
print(current_directory)


/content


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.2/107.7 GB disk)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load YOLOv8 Pose Model
model = YOLO("yolov8n-pose.pt")

# Check input video path
video_path = "/content/cow heating _vedio.mp4"
if not os.path.isfile(video_path):
    print(f"Error: Video not found at {video_path}")
    exit()

cap = cv2.VideoCapture(video_path)

# Check if the video is opened
if not cap.isOpened():
    print("Error: Cannot open input video!")
    exit()

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"XVID")  # More compatible codec

output_path = "/content/Cow_Heat_Detection.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Function to check mounting behavior
def is_mounted(cow1, cow2):
    try:
        back1_y = cow1[8][1]  # Back keypoint of cow1
        front_leg1_y = cow1[10][1]  # Front leg of cow1
        back2_y = cow2[8][1]  # Back keypoint of cow2
        front_leg2_y = cow2[10][1]  # Front leg of cow2

        # Mounting condition
        return back1_y < (back2_y - 40) and front_leg1_y < (front_leg2_y - 40)
    except IndexError:
        return False

# Process frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)
    cows = []

    # Collect cow keypoints
    for result in results:
        if result.keypoints is None:
            continue
        for keypoints in result.keypoints.xy.cpu().numpy():
            if len(keypoints) >= 13:
                cows.append(keypoints)

    # Detect mounting
    in_heat_count = 0
    mounted_pairs = set()
    for i, cow1 in enumerate(cows):
        for j, cow2 in enumerate(cows):
            if i != j and (i, j) not in mounted_pairs and is_mounted(cow1, cow2):
                in_heat_count += 1
                mounted_pairs.add((i, j))

                x, y = int(cow1[8][0]), int(cow1[8][1])
                cv2.putText(frame, "In Heat", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display the number of cows in heat
    cv2.putText(frame, f"Cows in Heat: {in_heat_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()

if os.path.isfile(output_path):
    print("Processing complete! Download the processed video here:")
    print(output_path)
else:
    print("Error: Output video not generated!")



0: 384x640 (no detections), 164.5ms
Speed: 6.2ms preprocess, 164.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 172.2ms
Speed: 5.4ms preprocess, 172.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 180.0ms
Speed: 5.1ms preprocess, 180.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 231.8ms
Speed: 5.3ms preprocess, 231.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 250.7ms
Speed: 5.0ms preprocess, 250.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 251.9ms
Speed: 4.7ms preprocess, 251.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 260.0ms
Speed: 5.2ms preprocess, 260.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 247.6ms
Speed: 4.9ms prepr

In [ ]:
from IPython.display import display, HTML

video_path = "/mnt/data/Cow_Heat_Detection.mp4"
display(HTML(f"""
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
    </video>
"""))


In [ ]:
from google.colab import files

output_path = "/content/Cow_Heat_Detection.mp4"

# Download the video
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>